In [10]:
# Import packages
import sys
import os

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

import tqdm

import torch
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

from IPython.display import clear_output
from IPython import display

%matplotlib inline

# check and use GPU if available if not use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch.distributions import MultivariateNormal, Normal


In [11]:
class ContinuousActor(nn.Module):
    def __init__(
        self, 
        in_dim: int, 
        out_dim: int, 
    ):
        """Initialize."""
        super(ContinuousActor, self).__init__()

        self.hidden = nn.Linear(in_dim, 32)
        self.mu_layer = nn.Linear(32, out_dim)
        self.log_std_layer = nn.Linear(32, out_dim)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        x = F.relu(self.hidden(state))
        
        mu = torch.tanh(self.mu_layer(x))
        log_std = torch.tanh(self.log_std_layer(x))

        std = torch.exp(log_std)
        dist = Normal(mu, std)
        action = dist.sample()

        return action, dist

In [12]:
# CODE from another notebook
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, output_size, activation, layers=[32,32,16]):
        super().__init__()

        # Define layers with ReLU activation
        self.linear1 = torch.nn.Linear(input_size, layers[0])
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(layers[0], layers[1])
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(layers[1], layers[2])
        self.activation3 = torch.nn.ReLU()

        self.output_layer = torch.nn.Linear(layers[2], output_size)
        self.output_activation = activation

        # Initialization using Xavier normal (a popular technique for initializing weights in NNs)
        torch.nn.init.xavier_normal_(self.linear1.weight)
        torch.nn.init.xavier_normal_(self.linear2.weight)
        torch.nn.init.xavier_normal_(self.linear3.weight)
        torch.nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, inputs):
        x = inputs
        # flatten the input
        # x = torch.flatten(x)
        # print(x)
        # # normalize the inputs for cartopole-v1
        # x[0] = x[0] / 4.8
        # x[2] = x[2] / 0.418
        # First normalize the inputs
        #x = (x - x.mean()) / x.std()

        # Then pass through the layers
        x = self.activation1(self.linear1(x))
        x = self.activation2(self.linear2(x))
        x = self.activation3(self.linear3(x))
        x = self.output_activation(self.output_layer(x))
        return x

# HERE ADDED ns, nA and discretized_continuous_action_space and actions_transform=None********************************************************************************
def generate_single_episode(env, policy_net, nS, nA, discretized_continuous_action_space, actions_transform=None):
    """
    Generates an episode by executing the current policy in the given env
    """
    
    states = []
    actions = []
    rewards = []
    log_probs = []
    max_t = 1000 # max horizon within one episode
    state, _ = env.reset()

    if isinstance(env.action_space, gym.spaces.Box): # continuous action space
        max_possible_action = float(env.action_space.high[0]) # Only works with a action space dim of 1
        min_possible_action = float(env.action_space.low[0]) # Only works with a action space dim of 1

    for t in range(max_t):
        #print(t)
        if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space: # HERE ADDED dicretized_continuous_action_space***********
            state = torch.from_numpy(state).float().unsqueeze(0)
        else:
            state = torch.from_numpy(state.flatten()).float()
            

        # if action space is discrete or continuous
        if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space: # HERE ADDED dicretized_continuous_action_space***********
            # normalize the state
            probs = policy_net.forward(Variable(state)) # get each action choice probability with the current policy network
            action = np.random.choice(nA, p=np.squeeze(probs.detach().numpy())) # probablistic ## HERE ADDED nA instead of env.action_space.n******************
        else:
            action, dist = policy_net.forward(state) # continuous
            # clip action to the action space
            action = torch.clamp(action, min=min_possible_action, max=max_possible_action)
            action = action.detach().numpy()
            probs = dist


        # compute the log_prob to use this in parameter update
        log_prob = None
        if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space: # HERE ADDED dicretized_continuous_action_space***********
            log_prob = torch.log(probs.squeeze(0)[action])
        else:
            log_prob = dist.log_prob(torch.tensor(action))
            # print(f"log_prob: {log_prob}")
            
        #print(log_prob)
        
        # append values
        states.append(state)
        actions.append(action)
        log_probs.append(log_prob)
        
        # take a selected action
        if isinstance(env.action_space, gym.spaces.Discrete):
            state, reward, terminated, truncated, _ = env.step(action)
            #reward = (reward + 8.1) / 8.1 # normalize reward only for pendulum
        # HERE ********************************************************************************************************************
        elif discretized_continuous_action_space:
            assert actions_transform is not None, "actions_transform must be provided for discretized_continuous_action_space" 
            state, reward, terminated, truncated, _ = env.step([actions_transform[action]])
            reward = (reward + 8.1) / 8.1 # normalize reward only for pendulum
        #**************************************************************************************************************************
        else:
            state, reward, terminated, truncated, _ = env.step(action)
            reward = (reward + 8.1) / 8.1 # normalize reward only for pendulum
        rewards.append(reward)

        if terminated | truncated:
            break

    # print(f"Episode length: {t}")
    # print(f"Episode reward: {np.sum(rewards)}")
    # print(f"Episode states: {states}")
    # print(f"Episode actions: {actions}")
    # print(f"Episode rewards: {rewards}")
    # print(f"Episode log_probs: {log_probs}")
    # print(f"Episode state", state)
    # print(f"Episode action trans", action)
    # print("-----------------------------------------------------------------------------")

            
    return states, actions, rewards, log_probs

# HERE ADDED ns, nA and discretized_continuous_action_space and actions_transform=None********************************************************************************
def evaluate_policy(env, policy_net, nS, nA, discretized_continuous_action_space, actions_transform=None):
    """
    Compute accumulative trajectory reward
    """
    states, actions, rewards, log_probs = generate_single_episode(env, policy_net, nS, nA, discretized_continuous_action_space, actions_transform) # HERE ADDED ns, nA and discretized_continuous_action_space and actions_transform************
    return np.sum(rewards)

In [13]:
# HERE ADDED nS, nA and discretized_continuous_action_space and actions_transform=None*********************************************************************************
def train_PPO(env, policy_net, policy_optimizer, value_net, value_optimizer, num_epochs, nS, nA, discretized_continuous_action_space, actions_transform = None, clip_val=0.2, gamma=0.99, entropy_coef=0.005, lamda=0.95):
    """
    Trains the policy network using PPO
    """

    # Generate an episode with the current policy network
    states, actions, rewards, log_probs = generate_single_episode(env, policy_net, nS, nA, discretized_continuous_action_space, actions_transform)
    T = len(states)
    
    # Create tensors depending on if it is discrete or continuous action space
    if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space:
        actions = torch.LongTensor(actions).to(device).view(-1,1)
    else:
        actions = torch.FloatTensor(actions).to(device).view(-1,1)


    states = np.vstack(states).astype(float)
    states = torch.FloatTensor(states).to(device)
    rewards = torch.FloatTensor(rewards).to(device).view(-1,1)
    log_probs = torch.FloatTensor(log_probs).to(device).view(-1,1)

    # Compute the generalized advantage estimate
    Gs = []
    G = 0
    for t in range(T-1,-1,-1):
        delta = (rewards[t] + gamma*value_net(states[t]) - value_net(states[t-1]))
        G = delta + gamma * G * lamda
        Gs.insert(0,G)
    Gs = torch.tensor(Gs).view(-1,1)
    #print(f"Gs: {Gs}")
    
    # for t in range(T-1,-1,-1): # iterate in backward order to make the computation easier
    #     G = rewards[t] + gamma*G
    #     Gs.insert(0,G)
    # Gs = torch.tensor(Gs).view(-1,1)
    
    # Compute the advantage
    state_vals = value_net(states).to(device)
    #print(f"State values: {state_vals}")
    with torch.no_grad():
        A_k = Gs - state_vals
        
    for t in range(num_epochs):
        # Compute the value of the current states
        V = value_net(states).to(device)
        #print(f"Value: {V}")

        # Calculate probability of each action under the updated policy
        # compute the log_prob to use it in parameter update
        if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space: # HERE ADDED dicretized_continuous_action_space***********
            probs = policy_net.forward(states).to(device)
            curr_log_probs = torch.log(torch.gather(probs, 1, actions))
            # print(f"probs, discrete: {probs}")
        else:
            _, probs = policy_net.forward(states)
            curr_log_probs = probs.log_prob(actions)
            # print(f"probs, continuous: {probs}")

        # Calculate ratios r(theta)
        ratios = torch.exp(curr_log_probs - log_probs)
        
        # Calculate two surrogate loss terms in cliped loss
        surr1 = ratios * A_k
        surr2 = torch.clamp(ratios, 1-clip_val, 1+clip_val) * A_k
        
        # entropy 
        # HERE ADDED dicretized_continuous_action_space AND reverse the not*************************************************************************************
        if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space: 
            # Calculate clipped loss value
            actor_loss = (-torch.min(surr1, surr2)).mean() # Need negative sign to run Gradient Ascent
        else:
            entropy = probs.entropy().mean()
            actor_loss = (-torch.min(surr1, surr2).mean() - entropy_coef * entropy)
        #****************************************************************************************************************************************************
        
        # Update policy network
        policy_optimizer.zero_grad()
        actor_loss.backward(retain_graph=True)
        policy_optimizer.step()
        
        # Update value net
        critic_loss = nn.MSELoss()(V, Gs)
        value_optimizer.zero_grad()
        critic_loss.backward()
        value_optimizer.step()

    # print(f"Value: {V}")
    # print(f"Advantage: {A_k}")
    # print(f"Ratios: {ratios}")
    # print(f"Surrogate 1: {surr1}")
    # print(f"Surrogate 2: {surr2}")
    # print(f"Actor loss: {actor_loss}")
    # print(f"Critic loss: {critic_loss}")
    # print("-----------------------------------------------------------------------------")
        
    return policy_net, value_net

In [15]:
# Define parameter values
#env_name = 'CartPole-v1' # environment name
env_name = 'Pendulum-v1' # environment name

#env_name = 'LunarLander-v2' # environment name
# env_name = 'MountainCar-v0' # environment name
# env_name = 'Acrobot-v1' # environment name


num_train_ite = 2000 # number of training iterations
num_seeds = 1 # fit model with 3 different seeds and plot average performance of 3 seeds
num_epochs = 10 # how many times we iterate the entire training dataset passing through the training
eval_freq = 50 # run evaluation of policy at each eval_freq trials
eval_epi_index = num_train_ite//eval_freq # use to create x label for plot
returns = np.zeros((num_seeds, eval_epi_index))
gamma = 0.99 # discount factor
clip_val = 0.2 # hyperparameter epsilon in clip objective
entropy_coef = 0.001 # hyperparameter entropy coefficient (0.005)

# Create the environment.
env = gym.make(env_name)



# Discretized Action Space**********************************************************************************************
discretized_continuous_action_space = False

if env_name == 'Pendulum-v1':
    discretized_continuous_action_space = True
    
actions_transform = None
if discretized_continuous_action_space:
    action_range = (env.action_space.low[0], env.action_space.high[0])
    n_actions = 11
    actions_transform = np.linspace(action_range[0], action_range[1], n_actions)

#************************************************************************************************************************

#detect if the environment is discrete or continuous
if isinstance(env.action_space, gym.spaces.Discrete):
    nA = env.action_space.n
else:
    # HERE ADDED dicretized_continuous_action_space********************************************************************
    if discretized_continuous_action_space:
        nA = n_actions
    else:
        nA = env.action_space.shape[0]
    #****************************************************************************************************************

# detect if the environment state is discrete or continuous
if isinstance(env.observation_space, gym.spaces.Discrete):
    nS = env.observation_space.n
else:
    nS = env.observation_space.shape[0]

policy_lr = 1e-4 # policy network's learning rate 
baseline_lr = 5e-4 # value network's learning rate

for i in tqdm.tqdm(range(num_seeds)):
    reward_means = []

    # Define policy and value networks
    if isinstance(env.action_space, gym.spaces.Discrete) or discretized_continuous_action_space: # HERE ADDED dicretized_continuous_action_space***********
        policy_net = NeuralNet(nS, nA, torch.nn.Softmax()) # Normally this one works with CartPole-v0
    else:
        policy_net = ContinuousActor(nS, nA)
    policy_net_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)
    value_net = NeuralNet(nS, 1, torch.nn.ReLU())
    value_net_optimizer = optim.Adam(value_net.parameters(), lr=baseline_lr)
    
    for m in range(num_train_ite):
        # Train networks with PPO # HERE ADDED nA, nS and discretized_continuous_action_space******************************************************************
        policy_net, value_net = train_PPO(
            env, policy_net, policy_net_optimizer, value_net, value_net_optimizer, num_epochs, 
            nS, nA, discretized_continuous_action_space, actions_transform=actions_transform,
            clip_val=clip_val, gamma=gamma, entropy_coef=entropy_coef)
        #********************************************************************************************************************************************************
        if m % eval_freq == 0:
            print("Episode: {}".format(m))
            G = np.zeros(20)
            for k in range(20):
                g = evaluate_policy(env, policy_net, nS, nA, discretized_continuous_action_space, actions_transform) # HERE ADDED nS, nA and discretized_continuous_action_space and actions_transform**************
                G[k] = g

            reward_mean = G.mean()
            reward_sd = G.std()
            print("The avg. test reward for episode {0} is {1} with std of {2}.".format(m, reward_mean, reward_sd))
            reward_means.append(reward_mean)
    returns[i] = np.array(reward_means)

# save the policy network
torch.save(policy_net.state_dict(), f"policy_net_{env_name}.pt")


# Plot the performance over iterations
x = np.arange(eval_epi_index)*eval_freq
avg_returns = np.mean(returns, axis=0)
max_returns = np.max(returns, axis=0)
min_returns = np.min(returns, axis=0)

plt.fill_between(x, min_returns, max_returns, alpha=0.1)
plt.plot(x, avg_returns, '-o', markersize=1)

plt.xlabel('Episode', fontsize = 15)
plt.ylabel('Return', fontsize = 15)

plt.title("PPO Learning Curve", fontsize = 24)

  0%|          | 0/3 [00:00<?, ?it/s]

Episode: 0
The avg. test reward for episode 0 is 53.81091809916482 with std of 30.181170769555386.
Episode: 50
The avg. test reward for episode 50 is 74.12877525171115 with std of 21.141939191265767.
Episode: 100
The avg. test reward for episode 100 is 75.90052526058597 with std of 26.32409788729959.
Episode: 150
The avg. test reward for episode 150 is 63.15925059535142 with std of 23.19549217105326.
Episode: 200
The avg. test reward for episode 200 is 43.31209688163818 with std of 35.34744831188236.
Episode: 250
The avg. test reward for episode 250 is 54.717379749439985 with std of 35.136319080713484.
Episode: 300
The avg. test reward for episode 300 is 65.02027820257841 with std of 25.903730024042247.
Episode: 350
The avg. test reward for episode 350 is 47.89018259343614 with std of 31.094756883753913.
Episode: 400
The avg. test reward for episode 400 is 68.4308915264641 with std of 25.69573958746214.
Episode: 450
The avg. test reward for episode 450 is 76.65590252053882 with std of 

 33%|███▎      | 1/3 [10:20<20:41, 620.55s/it]

Episode: 0
The avg. test reward for episode 0 is 54.785632885079146 with std of 32.05609890712485.
Episode: 50
The avg. test reward for episode 50 is 51.289915256339135 with std of 34.36918752239968.
Episode: 100
The avg. test reward for episode 100 is 59.0437434404296 with std of 29.405286116809783.
Episode: 150
The avg. test reward for episode 150 is 70.43010284691265 with std of 23.28484947120025.


 33%|███▎      | 1/3 [11:23<22:46, 683.47s/it]


KeyboardInterrupt: 

In [52]:
state

array([0.19139662, 0.9815128 , 5.5108323 ], dtype=float32)

In [ ]:
# save the policy network
torch.save(policy_net.state_dict(), f"policy_net_discrete_working{env_name}.pt")

In [15]:
# FOR  DISCETIZED CONTINUOUS ACTION SPACE****************************************************************************************
# THINGS HAVE CHANGED HERE TO DO IT FOR DISCRETIZED CONTINUOUS ACTION SPACE

import time
#create an environment to test and visualize the policy
#env_name = 'CartPole-v0' # environment name
env_name = 'Pendulum-v1' # environment name
# env_name = 'MountainCar-v0' # environment name
# env_name = 'Acrobot-v1' # environment name
env = gym.make(env_name, render_mode='human')

if isinstance(env.action_space, gym.spaces.Discrete):
    nS = env.observation_space.shape[0]
    nA = env.action_space.n
    policy_net = NeuralNet(nS, nA, torch.nn.Softmax())
elif discretized_continuous_action_space: #************************************************************************************
    nS = env.observation_space.shape[0]
    nA = n_actions
    policy_net = NeuralNet(nS, nA, torch.nn.Softmax())
else:
    nS = env.observation_space.shape[0]
    nA = env.action_space.shape[0]
    policy_net = ContinuousActor(nS, nA)

# load the policy network
policy_net.load_state_dict(torch.load(f"policy_net_{env_name}.pt"))
# policy_net.eval()

episodes = 1
state = env.reset()[0]
rewards = []
for i in range(episodes):
    while True:
        probs = policy_net.forward(torch.tensor(state, dtype=torch.float32))#************************************************************************************
        action = np.random.choice(nA, p=np.squeeze(probs.detach().numpy()))#************************************************************************************
        state, reward, done, truncated, _ = env.step([actions_transform[action]]) #************************************************************************************
        if done or truncated:
            break
        state = torch.from_numpy(state.flatten()).float()
        rewards.append(reward)
    print(f"Episode {i}: {np.sum(rewards)}")
    env.reset()
env.close()

C:\Temp\ipykernel_10680\1596193349.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probs = policy_net.forward(torch.tensor(state, dtype=torch.float32))#************************************************************************************


Episode 0: -906.0140322749731


In [17]:
import time
#create an environment to test and visualize the policy
#env_name = 'CartPole-v0' # environment name
#env_name = 'Pendulum-v1' # environment name
# env_name = 'MountainCar-v0' # environment name
# env_name = 'Acrobot-v1' # environment name
env = gym.make(env_name, render_mode='human')

if isinstance(env.action_space, gym.spaces.Discrete):
    nS = env.observation_space.shape[0]
    nA = env.action_space.n
    policy_net = NeuralNet(nS, nA, torch.nn.Softmax())
else:
    nS = env.observation_space.shape[0]
    nA = env.action_space.shape[0]
    policy_net = ContinuousActor(nS, nA)

# load the policy network
policy_net.load_state_dict(torch.load(f"policy_net_{env_name}.pt"))
# policy_net.eval()

episodes = 1
state = env.reset()[0]
rewards = []
for i in range(episodes):
    while True:
        action, _ = policy_net.forward(torch.tensor(state, dtype=torch.float32))
        # clip action to the action space
        action = torch.clamp(action, min=-2, max=2)
        
        # take a selected action
        action = action.detach().numpy()
        state, reward, done, truncated, _ = env.step(action)
        if done or truncated:
            break
        state = torch.from_numpy(state.flatten()).float()
        rewards.append(reward)
    print(f"Episode {i}: {np.sum(rewards)}")
    env.reset()
env.close()

C:\Temp\ipykernel_13460\2025769064.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action, _ = policy_net.forward(torch.tensor(state, dtype=torch.float32))


Episode 0: -1347.13148178803


In [16]:
env.close()